# Review Classification v2
## Fine-tuning Yelp-pretrained RoBERTa on Amazon Reviews
---
**Base model:** RoBERTa fine-tuned on Yelp reviews (Felipe's checkpoint-1000)

**Improvements over v1:**
1. Stratified train/test split on our Amazon data (80/20)
2. Class weights to handle 90/5.6/4.3 imbalance
3. Fine-tuning on our actual Amazon data (not just Yelp)
4. Early stopping to prevent overfitting
5. Proper held-out evaluation

## 1. Imports

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    confusion_matrix, classification_report,
    accuracy_score, precision_recall_fscore_support
)
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, DataCollatorWithPadding,
    EarlyStoppingCallback
)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Load and Prepare Data

In [ ]:
df = pd.read_csv("data_cleaned.csv")
print(f"Dataset: {len(df):,} rows")
print(f"\nSentiment distribution:")
print(df["sentiment"].value_counts())

In [ ]:
# Map sentiment to numeric labels
label_map = {"NEGATIVE": 0, "NEUTRAL": 1, "POSITIVE": 2}
id2label = {0: "NEGATIVE", 1: "NEUTRAL", 2: "POSITIVE"}
label2id = {v: k for k, v in id2label.items()}

df["label"] = df["sentiment"].str.upper().map(label_map)

# Verify mapping
print("Label mapping check:")
print(df.groupby(["sentiment", "label"]).size())

## 3. Stratified Train/Test Split

In [ ]:
train_df, test_df = train_test_split(
    df, 
    test_size=0.2, 
    random_state=42, 
    stratify=df["label"]
)

print(f"Train: {len(train_df):,} rows")
print(f"Test:  {len(test_df):,} rows")

print(f"\nTrain distribution:")
print(train_df["label"].value_counts(normalize=True).round(4))
print(f"\nTest distribution:")
print(test_df["label"].value_counts(normalize=True).round(4))

## 4. Compute Class Weights

In [ ]:
# Compute balanced class weights from training data
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0, 1, 2]),
    y=train_df["label"].values
)

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

print("Class weights:")
for i, w in enumerate(class_weights):
    print(f"  {id2label[i]}: {w:.4f}")

print(f"\nInterpretation: Negative errors cost {class_weights[0]:.1f}x, "
      f"Neutral errors cost {class_weights[1]:.1f}x, "
      f"Positive errors cost {class_weights[2]:.1f}x")

## 5. Tokenize Data

In [ ]:
# Load tokenizer from Felipe's Yelp-pretrained model
# UPDATE THIS PATH to point to your Yelp-trained checkpoint
YELP_MODEL_PATH = "./models/yelp_roberta_3class/checkpoint-1000"

tokenizer = AutoTokenizer.from_pretrained(YELP_MODEL_PATH)
print(f"Tokenizer loaded from: {YELP_MODEL_PATH}")

In [ ]:
# Convert to HuggingFace datasets
train_dataset = Dataset.from_pandas(train_df[["reviews.text", "label"]].reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df[["reviews.text", "label"]].reset_index(drop=True))

def tokenize_fn(batch):
    return tokenizer(
        batch["reviews.text"],
        truncation=True,
        padding=False,  # let DataCollator handle padding (more efficient)
        max_length=256
    )

train_tokenized = train_dataset.map(tokenize_fn, batched=True, remove_columns=["reviews.text"])
test_tokenized = test_dataset.map(tokenize_fn, batched=True, remove_columns=["reviews.text"])

# Rename label to labels (HuggingFace convention)
train_tokenized = train_tokenized.rename_column("label", "labels")
test_tokenized = test_tokenized.rename_column("label", "labels")

train_tokenized.set_format("torch")
test_tokenized.set_format("torch")

print(f"Train tokenized: {len(train_tokenized)} samples")
print(f"Test tokenized:  {len(test_tokenized)} samples")
print(f"Features: {train_tokenized.column_names}")

## 6. Load Yelp-Pretrained Model

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    YELP_MODEL_PATH,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model loaded from: {YELP_MODEL_PATH}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

## 7. Custom Trainer with Class Weights

In [ ]:
class WeightedTrainer(Trainer):
    """Custom Trainer that applies class weights to the loss function."""
    
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        loss_fn = torch.nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fn(logits, labels)
        
        return (loss, outputs) if return_outputs else loss

print("WeightedTrainer defined.")
print(f"Class weights that will be used: {class_weights_tensor}")

## 8. Training Configuration

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="macro", zero_division=0
    )
    precision_w, recall_w, f1_w, _ = precision_recall_fscore_support(
        labels, preds, average="weighted", zero_division=0
    )
    
    return {
        "accuracy": acc,
        "f1_macro": f1,
        "precision_macro": precision,
        "recall_macro": recall,
        "f1_weighted": f1_w,
        "precision_weighted": precision_w,
        "recall_weighted": recall_w,
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./models/amazon_roberta_v2",
    
    # Training
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    
    # Evaluation & saving
    eval_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    logging_steps=50,
    
    # Early stopping
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",  # optimize for macro F1 (balances all classes)
    greater_is_better=True,
    save_total_limit=3,
    
    # Performance
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
    report_to="none",
)

print("Training arguments configured.")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Best model metric: {training_args.metric_for_best_model}")

## 9. Train

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = WeightedTrainer(
    class_weights=class_weights_tensor,
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print(f"Training on {len(train_tokenized):,} samples, evaluating on {len(test_tokenized):,} samples")
print(f"Early stopping patience: 3 eval steps without improvement")
print("\nStarting training...")

train_result = trainer.train()

In [ ]:
# Training summary
print(f"Training completed in {train_result.metrics['train_runtime']:.0f} seconds")
print(f"Final training loss: {train_result.metrics['train_loss']:.4f}")
print(f"Total steps: {train_result.global_step}")

## 10. Evaluate on Test Set

In [ ]:
# Get predictions on test set
predictions = trainer.predict(test_tokenized)
y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=-1)

# Classification report
labels = [id2label[i] for i in range(3)]
print("=" * 60)
print("CLASSIFICATION REPORT (Amazon Test Set)")
print("=" * 60)
print(classification_report(y_true, y_pred, target_names=labels, digits=4))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Raw counts
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=labels, yticklabels=labels, ax=axes[0])
axes[0].set_title("Confusion Matrix (counts)")
axes[0].set_ylabel("True Label")
axes[0].set_xlabel("Predicted Label")

# Normalized by true label (recall per class)
cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_norm, annot=True, fmt=".2%", cmap="Blues",
            xticklabels=labels, yticklabels=labels, ax=axes[1])
axes[1].set_title("Confusion Matrix (normalized by true label)")
axes[1].set_ylabel("True Label")
axes[1].set_xlabel("Predicted Label")

plt.tight_layout()
plt.show()

## 11. Compare v1 vs v2

In [ ]:
# v1 results (from Felipe's notebook - on full dataset, no train/test split)
v1_results = {
    "Accuracy": 0.8723,
    "F1 Macro": 0.6347,
    "Negative F1": 0.6985,
    "Neutral F1": 0.2689,
    "Positive F1": 0.9367
}

# v2 results (from this notebook - on held-out test set)
report = classification_report(y_true, y_pred, target_names=labels, digits=4, output_dict=True)
v2_results = {
    "Accuracy": report["accuracy"],
    "F1 Macro": report["macro avg"]["f1-score"],
    "Negative F1": report["NEGATIVE"]["f1-score"],
    "Neutral F1": report["NEUTRAL"]["f1-score"],
    "Positive F1": report["POSITIVE"]["f1-score"]
}

comparison = pd.DataFrame({"v1 (Yelp only)": v1_results, "v2 (+ Amazon fine-tune)": v2_results})
comparison["Change"] = comparison["v2 (+ Amazon fine-tune)"] - comparison["v1 (Yelp only)"]
print("Model Comparison:")
print("Note: v1 was evaluated on full dataset (no split), v2 on held-out test set")
print(comparison.round(4).to_string())

## 12. Error Analysis

In [ ]:
# Look at misclassified reviews
test_df_eval = test_df.reset_index(drop=True).copy()
test_df_eval["predicted"] = [id2label[p] for p in y_pred]
test_df_eval["true"] = [id2label[t] for t in y_true]
test_df_eval["correct"] = test_df_eval["predicted"] == test_df_eval["true"]

errors = test_df_eval[~test_df_eval["correct"]]
print(f"Total errors: {len(errors)} / {len(test_df_eval)} ({len(errors)/len(test_df_eval)*100:.1f}%)")
print(f"\nError breakdown:")
print(errors.groupby(["true", "predicted"]).size().sort_values(ascending=False))

In [ ]:
# Sample misclassified reviews by type
print("=" * 60)
print("SAMPLE ERRORS: Positive predicted as Negative")
print("=" * 60)
subset = errors[(errors["true"] == "POSITIVE") & (errors["predicted"] == "NEGATIVE")]
for _, row in subset.head(5).iterrows():
    print(f"  Rating: {row['reviews.rating']} | Text: {str(row['reviews.text'])[:150]}")
    print()

print("=" * 60)
print("SAMPLE ERRORS: Negative predicted as Positive")
print("=" * 60)
subset = errors[(errors["true"] == "NEGATIVE") & (errors["predicted"] == "POSITIVE")]
for _, row in subset.head(5).iterrows():
    print(f"  Rating: {row['reviews.rating']} | Text: {str(row['reviews.text'])[:150]}")
    print()

print("=" * 60)
print("SAMPLE ERRORS: Neutral misclassified")
print("=" * 60)
subset = errors[errors["true"] == "NEUTRAL"]
for _, row in subset.head(5).iterrows():
    print(f"  Rating: {row['reviews.rating']} | Predicted: {row['predicted']} | Text: {str(row['reviews.text'])[:150]}")
    print()

## 13. Save Final Model

In [ ]:
# Save the best model
FINAL_MODEL_PATH = "./models/amazon_roberta_v2_final"

trainer.save_model(FINAL_MODEL_PATH)
tokenizer.save_pretrained(FINAL_MODEL_PATH)

print(f"Model saved to: {FINAL_MODEL_PATH}")

In [ ]:
# Quick test with the saved model
from transformers import pipeline

clf = pipeline(
    "text-classification",
    model=FINAL_MODEL_PATH,
    tokenizer=FINAL_MODEL_PATH,
    device=device,
    top_k=None
)

test_reviews = [
    "This product is terrible, waste of money.",
    "It's okay, nothing special but does the job.",
    "Absolutely love it! Best purchase I've made.",
    "good",
    "Batteries died after one week. Very disappointed.",
    "Works as expected for the price.",
]

print("Quick model test:")
print("-" * 60)
for review in test_reviews:
    result = clf(review)
    top = max(result, key=lambda x: x["score"])
    print(f"  [{top['label']:>8s} {top['score']:.2%}] {review}")

## 14. Generate Predictions for Full Dataset

Run the model on all reviews and save for use in clustering and summarization notebooks.

In [ ]:
# Predict on full dataset
all_texts = df["reviews.text"].fillna("").astype(str).tolist()

print(f"Running predictions on {len(all_texts):,} reviews...")
all_preds = clf(all_texts, batch_size=64, truncation=True, max_length=256)
print("Done!")

In [ ]:
# Extract predictions and confidence scores
df["predicted_label"] = [max(item, key=lambda x: x["score"])["label"] for item in all_preds]
df["predicted_score"] = [max(item, key=lambda x: x["score"])["score"] for item in all_preds]

def get_score(item, label):
    for i in item:
        if i["label"] == label:
            return i["score"]
    return None

df["score_negative"] = [get_score(item, "NEGATIVE") for item in all_preds]
df["score_neutral"] = [get_score(item, "NEUTRAL") for item in all_preds]
df["score_positive"] = [get_score(item, "POSITIVE") for item in all_preds]

# Save
df.to_csv("data_with_predictions_v2.csv", index=False)
print(f"Saved predictions to data_with_predictions_v2.csv")

print(f"\nPrediction distribution:")
print(df["predicted_label"].value_counts())

print(f"\nTrue vs Predicted:")
true_col = df["sentiment"].str.upper()
pred_col = df["predicted_label"]
print(classification_report(true_col, pred_col, target_names=labels, digits=4))

## 15. Summary

**Model:** RoBERTa-base → fine-tuned on Yelp 3-class → fine-tuned on Amazon reviews with class weights

**Key improvements over v1:**
- Class weights penalize errors on minority classes (Negative ~6x, Neutral ~8x vs Positive)
- Trained on actual Amazon data, not just Yelp transfer
- Proper stratified 80/20 train/test split
- Early stopping prevents overfitting (patience=3 on macro F1)
- Optimized for macro F1 instead of accuracy (better for imbalanced data)

**Files produced:**
- `./models/amazon_roberta_v2_final/` — saved model for deployment
- `data_with_predictions_v2.csv` — full dataset with predicted labels and confidence scores